In [72]:
#imports
from rply import LexerGenerator
from rply.token import BaseBox

In [73]:
lg = LexerGenerator()

lg.add('NUMERO', r'\d+')
lg.add('MAIS', r'mais')
lg.add('MENOS', r'menos')
lg.add('MULTIPLICACAO', r'multiplicado_por')
lg.add('DIVISAO', r'dividido_por')
lg.add('ABREPARENTESES', r'\(')
lg.add('FECHAPARENTESES', r'\)')

lg.add('IGUAL', r'recebe')
lg.add('INTEIRO', r'inteiro')
lg.add('TEXTO', r'texto')
lg.add('PONTOVIRGULA', r'\;')

#adicionar if
lg.add('SE', r'se')
lg.add('FAZ', r'faz')
lg.add('CASO_CONTRARIO', r'caso_contrario_faz')
lg.add('ENQUANTO', r'enquanto')
lg.add('COMPARADOR','for_igual_a')
lg.add('COMPARADOR','for_diferente_de')
lg.add('COMPARADOR','for_maior_ou_igual_a')
lg.add('COMPARADOR','for_menor_ou_igual_a')
lg.add('COMPARADOR','for_maior_que')
lg.add('COMPARADOR','for_menor_que')
lg.add('VARIAVEL', r'[a-zA-z][a-zA-z0-9]*')


lg.ignore('\s+')
lexer = lg.build()

In [74]:
tokens = lexer.lex("x gustavo y inteiro x recebe 10 y recebe 20")
for token in tokens:
    print(token)

Token('VARIAVEL', 'x')
Token('VARIAVEL', 'gustavo')
Token('VARIAVEL', 'y')
Token('INTEIRO', 'inteiro')
Token('VARIAVEL', 'x')
Token('IGUAL', 'recebe')
Token('NUMERO', '10')
Token('VARIAVEL', 'y')
Token('IGUAL', 'recebe')
Token('NUMERO', '20')


In [75]:
# #ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITOR

# class Programa(BaseBox):
#     def __init__(self, declaracoes,atribuicoes):
#         self.declaracoes = declaracoes
#         self.atribuicoes = atribuicoes

#     def accept(self, visitor):
#         visitor.visit_programa(self)

# class VarDeclaracoes(BaseBox):
#     def __init__(self, declaracao,declaracoes):
#         self.declaracao = declaracao
#         self.declaracoes = declaracoes

#     def accept(self, visitor):
#         visitor.visit_vardeclaracoes(self)

# class VarDeclaracao(BaseBox):
#     def __init__(self, variavel,tp):
#         self.variavel = variavel
#         self.tp = tp

#     def accept(self, visitor):
#         visitor.visit_vardeclaracao(self)

# class Atribuicoes(BaseBox):
#     def __init__(self, atribuicao,atribuicoes):
#         self.atribuicao = atribuicao
#         self.atribuicoes = atribuicoes

#     def accept(self, visitor):
#         visitor.visit_atribuicoes(self)

# class Atribuicao(BaseBox):
#     def __init__(self,expr):
#         self.expr = expr

#     def accept(self, visitor):
#         visitor.visit_atribuicao(self)

# class Atrib(BaseBox):
#     def __init__(self, variavel,expr):
#         self.variavel = variavel
#         self.expr = expr

#     def accept(self, visitor):
#         visitor.visit_atrib(self)

# class SeSenao(BaseBox):
#     def __init__(self, expr1, comp, expr2, corpo1,corpo2):
#         self.expr1=expr1
#         self.comp = comp
#         self.expr2=expr2
#         self.corpo1=corpo1
#         self.corpo2=corpo2

#     def accept(self, visitor):
#         visitor.visit_sesenao(self)


# class Enquanto(BaseBox):
#     def __init__(self, expr1, comp, expr2, corpo1):
#         self.expr1=expr1
#         self.comp = comp
#         self.expr2=expr2
#         self.corpo1=corpo1


#     def accept(self, visitor):
#         visitor.visit_enquanto(self)

# class ParaCada(BaseBox):
#     def __init__(self, variavelinic, exprinic, expr1, comp, expr2, variavelincr, exprincr, corpo1):
#         self.variavelinic=variavelinic
#         self.exprinic=exprinic
#         self.expr1=expr1
#         self.comp = comp
#         self.expr2=expr2
#         self.variavelincr=variavelincr
#         self.exprincr=exprincr
#         self.corpo1=corpo1


#     def accept(self, visitor):
#         visitor.visit_paracada(self)

# class Expr(BaseBox):
#     def accept(self, visitor):
#         method_name = 'visit_{}'.format(self.__class__.__name__.lower())
#         visit = getattr(visitor, method_name)
#         visit(self)

# class Variavel(Expr):
#     def __init__(self, valor):
#         self.valor = valor

# class Numero(Expr):
#     def __init__(self, valor):
#         self.valor = valor


# class OpBinario(Expr):
#     def __init__(self, esquerda, direita):
#         self.esquerda = esquerda
#         self.direita = direita

# class Soma(OpBinario):
#   pass


# class Subtracao(OpBinario):
#   pass


# class Multiplicacao(OpBinario):
#   pass


# class Divisao(OpBinario):
#   pass

In [76]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp


    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class For(BaseBox):
    def __init__(self, idinic, exprinic, expr1, comp, expr2, idincr, exprincr, ie1):
        self.idinic=idinic
        self.exprinic=exprinic
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.idincr=idincr
        self.exprincr=exprincr
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_for(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass

In [77]:
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMERO', 'MAIS', 'MENOS', 'MULTIPLICACAO', 'DIVISAO', 'ABREPARENTESES', 'FECHAPARENTESES','IGUAL','INTEIRO','TEXTO','PONTOVIRGULA','SE','FAZ','CASO_CONTRARIO','ENQUANTO','COMPARADOR','VARIAVEL'],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['MAIS', 'MENOS']),
        ('left', ['MULTIPLICACAO', 'DIVISAO'])
    ]
)

@pg.production('prog : vardecls')
def prog(p):
    return Prog(p[0],None)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : VARIAVEL TEXTO PONTOVIRGULA')
def vardecl_string(p):
    return VarDecl(p[0].getstr(), "TEXTO")

@pg.production('vardecl : VARIAVEL INTEIRO PONTOVIRGULA')
def vardecl_int(p):
    return VarDecl(p[0].getstr(), "INTEIRO")

##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : VARIAVEL IGUAL expression PONTOVIRGULA')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : SE ABREPARENTESES expression COMPARADOR expression FECHAPARENTESES FAZ openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[7],None)


@pg.production('openstatement : SE ABREPARENTESES expression COMPARADOR expression FECHAPARENTESES FAZ closedstatement CASO_CONTRARIO openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[7],p[9])

@pg.production('openstatement : ENQUANTO ABREPARENTESES expression COMPARADOR expression FECHAPARENTESES FAZ openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[7])


##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closedstatement : VARIAVEL IGUAL expression PONTOVIRGULA')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : SE ABREPARENTESES expression COMPARADOR expression FECHAPARENTESES FAZ closedstatement CASO_CONTRARIO closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[7],p[9])

@pg.production('closedstatement : ENQUANTO ABREPARENTESES expression COMPARADOR expression FECHAPARENTESES FAZ closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[7])


@pg.production('expression : VARIAVEL')
def expression_VARIAVEL(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMERO')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : ABREPARENTESES expression FECHAPARENTESES')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression MAIS expression')
@pg.production('expression : expression MENOS expression')
@pg.production('expression : expression MULTIPLICACAO expression')
@pg.production('expression : expression DIVISAO expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'MAIS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MENOS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MULTIPLICACAO':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIVISAO':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

C:\Users\Gustavin\AppData\Local\Temp\ipykernel_27808\3145348602.py:121: ParserGeneratorWarning: 1 shift/reduce conflict
  parser = pg.build()


In [78]:
# VISITOR PARA TABELA DE SÍMBOLOS

ST={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp

In [79]:
arvore=parser.parse(lexer.lex('x INTEIRO;y TEXTO; x=5;'))
arvore.accept(SymbolTable())
print(ST)

ParsingError: (None, SourcePosition(idx=2, lineno=1, colno=3))